In [3]:
import qiskit.extensions.simulator  
from qiskit import *

import os 
import sys 
import unittest
import logging
#import Qconfig
#import QconfigQobj
import QConfigCore
import QConfigStaging
from qiskit.backends.ibmq import IBMQProvider
#register(Qconfig.APItoken)
#register(QConfigQobj.APItoken, QConfigQobj.config['url'])
core_provider = IBMQProvider(QConfigCore.APItoken, QConfigCore.config['url'])
#unregister('ibmq_qasm_simulator')
staging_provider = IBMQProvider(QConfigStaging.APItoken, QConfigStaging.config['url'])
available_backends() # this is all the backends i have access to

logger = logging.Logger('error')
sys.path.append('../')

In [4]:
core_provider.backends

{'ibmq_1_atlantis': <qiskit.backends.ibmq.ibmqbackend.IBMQBackend at 0xa167cd390>,
 'ibmq_qasm_simulator': <qiskit.backends.ibmq.ibmqbackend.IBMQBackend at 0xa167cd470>}

In [5]:
qiskit.registered_providers()

In [6]:
available_backends()

['local_qasm_simulator',
 'local_statevector_simulator',
 'local_unitary_simulator']

In [7]:
backends = ['ibmqx_hpc_qasm_simulator','ibmq_1_atlantis','ibmq_qasm_simulator'] 

In [13]:
from qiskit.backends.ibmq import IBMQProvider
from python.common import Path, QiskitTestCase, requires_qe_access, slow_test


class TestBackendQobj(QiskitTestCase):
    """Qiskit backend qobj test."""
    
    def __init__(self, backend, *args, **kwargs):
        self.backend = backend
        super(TestBackendQobj, self).__init__(*args, **kwargs)
    
    def test_operational(self):
        local = get_backend('local_qasm_simulator')
        remote = self.backend
        self.assertTrue(remote.status()['operational'])
    
    def test_allow_qobj(self):
        local = get_backend('local_qasm_simulator')
        remote = self.backend
        self.assertTrue(remote.configuration()['allow_q_object'])
    def test_one_qubit_no_operation(self):
        """Test one circuit, one register, in-order readout.
        """
        qr = QuantumRegister(1)
        cr = ClassicalRegister(1)
        circ = QuantumCircuit(qr, cr)
        circ.measure(qr[0], cr[0])
        local = get_backend('local_qasm_simulator')
        remote = self.backend
        qobj = compile(circ, remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)
        
    def test_one_qubit_operation(self):
        """Test one circuit, one register, in-order readout.
        """
        qr = QuantumRegister(1)
        cr = ClassicalRegister(1)
        circ = QuantumCircuit(qr, cr)
        circ.x(qr[0])
        circ.measure(qr[0], cr[0])
        local = get_backend('local_qasm_simulator')
        remote = self.backend
        qobj = compile(circ, remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)
        
    def test_simple_circuit(self):
        """Test one circuit, one register, in-order readout.
        """
        qr = QuantumRegister(4)
        cr = ClassicalRegister(4)
        circ = QuantumCircuit(qr, cr)
        circ.x(qr[0])
        circ.x(qr[2])
        circ.measure(qr[0], cr[0])
        circ.measure(qr[1], cr[1])
        circ.measure(qr[2], cr[2])
        circ.measure(qr[3], cr[3])
        local = get_backend('local_qasm_simulator')
        remote = self.backend
        qobj = compile(circ, remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)


    def test_readout_order(self):
        """Test one circuit, one register, out-of-order readout.
        """
        qr = QuantumRegister(4)
        cr = ClassicalRegister(4)
        circ = QuantumCircuit(qr, cr)
        #circ.x(qr[0])
        #circ.x(qr[2])
        circ.measure(qr[0], cr[2])
        circ.measure(qr[1], cr[0])
        circ.measure(qr[2], cr[1])
        circ.measure(qr[3], cr[3])
        local = get_backend('local_qasm_simulator')
        remote = self.backend
        qobj_remote = compile(circ, remote)
        qobj_local = compile(circ, local)
        result_remote = remote.run(qobj_remote).result()
        result_local = local.run(qobj_local).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)

                
    def test_multi_register(self):
        """Test one circuit, two registers, out-of-order readout.
        """
        qr1 = QuantumRegister(2)
        qr2 = QuantumRegister(2)
        cr1 = ClassicalRegister(3)
        cr2 = ClassicalRegister(1)
        circ = QuantumCircuit(qr1, qr2, cr1, cr2)
        circ.h(qr1[0])
        circ.cx(qr1[0], qr2[1])
        circ.h(qr2[0])
        circ.cx(qr2[0], qr1[1])
        circ.x(qr1[1])
        circ.measure(qr1[0], cr2[0])
        circ.measure(qr1[1], cr1[0])
        circ.measure(qr1[1], cr2[0])
        circ.measure(qr1[1], cr1[2])
        circ.measure(qr2[0], cr1[2])
        circ.measure(qr2[1], cr1[1])
        local = get_backend('local_qasm_simulator')
        remote = self.backend
        qobj = compile(circ, remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)

    def test_multi_circuit(self):
        """Test two circuits, two registers, out-of-order readout.
        """
        qr1 = QuantumRegister(2)
        qr2 = QuantumRegister(2)
        cr1 = ClassicalRegister(3)
        cr2 = ClassicalRegister(1)
        circ1 = QuantumCircuit(qr1, qr2, cr1, cr2)
        circ1.h(qr1[0])
        circ1.cx(qr1[0], qr2[1])
        circ1.h(qr2[0])
        circ1.cx(qr2[0], qr1[1])
        circ1.x(qr1[1])
        circ1.measure(qr1[0], cr2[0])
        circ1.measure(qr1[1], cr1[0])
        circ1.measure(qr1[0], cr2[0])
        circ1.measure(qr1[1], cr1[2])
        circ1.measure(qr2[0], cr1[2])
        circ1.measure(qr2[1], cr1[1])
        circ2 = QuantumCircuit(qr1, qr2, cr1)
        circ2.h(qr1[0])
        circ2.cx(qr1[0], qr1[1])
        circ2.h(qr2[1])
        circ2.cx(qr2[1], qr1[1])
        circ2.measure(qr1[0], cr1[0])
        circ2.measure(qr1[1], cr1[1])
        circ2.measure(qr1[0], cr1[2])
        circ2.measure(qr2[1], cr1[2])
        local = get_backend('local_qasm_simulator')
        remote = self.backend

        qobj = compile([circ1, circ2], remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ1),
                              result_local.get_counts(circ1),delta=50)
        self.assertDictAlmostEqual(result_remote.get_counts(circ2),
                              result_local.get_counts(circ2),delta=50)
    
    def test_conditional_operation(self):
        """Test conditional operation. 
        """
        qr = QuantumRegister(4)
        cr = ClassicalRegister(4)
        circ = QuantumCircuit(qr, cr)
        circ.x(qr[0])
        circ.x(qr[2])
        circ.measure(qr[0], cr[0])
        circ.x(qr[0]).c_if(cr,1)
        local = get_backend('local_qasm_simulator')
        remote = self.backend

        qobj = compile(circ, remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)
    
    def test_erick_circuit(self):
        """Test one circuit, one register, in-order readout.
        """
        qr = QuantumRegister(3)
        cr = ClassicalRegister(3)
        circ= QuantumCircuit(qr, cr)
        circ.x(qr[0])
        circ.x(qr[2])
        circ.measure(qr, cr)
        local = get_backend('local_qasm_simulator')
        remote = self.backend
        qobj = compile(circ, remote)
        import pdb;pdb.set_trace()
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)

In [17]:
def test_backend(backend,name=None):
    if name is None:
        name = str(backend)
    print("Testing: {}".format(name))
    testbackend = TestBackendQobj(backend)
    test_methods = [testbackend.test_erick_circuit, testbackend.test_allow_qobj,testbackend.test_operational,testbackend.test_one_qubit_operation,
                    testbackend.test_one_qubit_no_operation,testbackend.test_multi_circuit,testbackend.test_multi_register,
               testbackend.test_readout_order,testbackend.test_simple_circuit,testbackend.test_conditional_operation]

    failed = 0 
    for test in test_methods:
        try:
            test()
            print('.',end='')
        except Exception as err:
            failed += 1
            print('x',end='')
            logger.warning(err)
            logger.error('test failed:{}'.format(str(test)))
    print('\n {0} of {1} tests failed'.format(failed,len(test_methods)))

In [18]:
core_provider.backends

{'ibmq_1_atlantis': <qiskit.backends.ibmq.ibmqbackend.IBMQBackend at 0xa167cd390>,
 'ibmq_qasm_simulator': <qiskit.backends.ibmq.ibmqbackend.IBMQBackend at 0xa167cd470>}

In [9]:
#ibmq atlantis
name = 'ibmq_1_atlantis'
backend = core_provider.backends[name]
test_backend(backend,name)

Testing: ibmq_1_atlantis


('0x4': 400 != 0), ('0x0': 560 != 0), ('101': 0 != 1024) within 50 delta
test failed:<bound method TestBackendQobj.test_erick_circuit of <__main__.TestBackendQobj testMethod=runTest>>


x..

('0x0': 991 != 0), ('1': 0 != 1024) within 50 delta
test failed:<bound method TestBackendQobj.test_one_qubit_operation of <__main__.TestBackendQobj testMethod=runTest>>


x

...

x

('0x0': 152 != 0), ('0x8': 173 != 0), ('0xA': 232 != 0), ('0x2': 152 != 0), ('0000': 0 != 1024) within 50 delta
test failed:<bound method TestBackendQobj.test_readout_order of <__main__.TestBackendQobj testMethod=runTest>>


x

('0x4': 72 != 0), ('0xC': 502 != 0), ('0x0': 94 != 0), ('0x8': 280 != 0), ('0101': 0 != 1024) within 50 delta
test failed:<bound method TestBackendQobj.test_simple_circuit of <__main__.TestBackendQobj testMethod=runTest>>
internal error
test failed:<bound method TestBackendQobj.test_conditional_operation of <__main__.TestBackendQobj testMethod=runTest>>


xx
 8 of 10 tests failed


In [10]:
#ibmq ibmq qasm sim 
name = 'ibmq_qasm_simulator'
backend = core_provider.backends[name]
test_backend(backend,name)

Testing: ibmq_qasm_simulator


False is not true
test failed:<bound method TestBackendQobj.test_allow_qobj of <__main__.TestBackendQobj testMethod=runTest>>


.x........
 1 of 10 tests failed


In [11]:
staging_provider.backends

{'ibmqx_hpc_qasm_simulator': <qiskit.backends.ibmq.ibmqbackend.IBMQBackend at 0xa20e01c88>,
 'ibmq_qasm_simulator': <qiskit.backends.ibmq.ibmqbackend.IBMQBackend at 0xa20e01160>}

In [12]:
#hpc staging
name = 'ibmqx_hpc_qasm_simulator'
backend = staging_provider.backends[name]
test_backend(backend,name)

Testing: ibmqx_hpc_qasm_simulator


KeyboardInterrupt: 

In [ ]:
#hpc staging
name = 'ibmq_qasm_simulator'
backend = staging_provider.backends[name]
test_backend(backend,name)

Testing: ibmq_qasm_simulator
> <ipython-input-13-e853567967aa>(193)test_erick_circuit()
-> result_remote = remote.run(qobj).result()
(Pdb) qobj
Qobj(config=QobjConfig(max_credits=10, memory_slots=3, n_qubits=3, shots=1024), experiments=[QobjExperiment(config=QobjItem(basis_gates='u1,u2,u3,cx,id', coupling_map='all-to-all', layout=None, memory_slots=3, n_qubits=3), header=QobjItem(clbit_labels=[['c3', 3]], compiled_circuit_qasm='OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q3[3];\ncreg c3[3];\nmeasure q3[1] -> c3[1];\nu3(3.14159265358979,0,3.14159265358979) q3[2];\nmeasure q3[2] -> c3[2];\nu3(3.14159265358979,0,3.14159265358979) q3[0];\nmeasure q3[0] -> c3[0];\n', name='circuit4', number_of_clbits=3, number_of_qubits=3, qubit_labels=[['q3', 0], ['q3', 1], ['q3', 2]]), instructions=[QobjItem(name='u3', params=[3.141592653589793, 0.0, 3.141592653589793], qubits=[2], texparams=['\\pi', '0', '\\pi']), QobjItem(clbits=[2], memory=[2], name='measure', qubits=[2]), QobjItem(clbits=[1], memory=[1

(Pdb) json.dumps(qd)
'{"qobj_id": "ba3ca765-b36b-468c-9704-7051feadfe67", "config": {"shots": 1024, "memory_slots": 3, "max_credits": 10, "n_qubits": 3}, "experiments": [{"instructions": [{"name": "u3", "params": [3.141592653589793, 0.0, 3.141592653589793], "texparams": ["\\\\pi", "0", "\\\\pi"], "qubits": [2]}, {"name": "measure", "qubits": [2], "clbits": [2], "memory": [2]}, {"name": "measure", "qubits": [1], "clbits": [1], "memory": [1]}, {"name": "u3", "params": [3.141592653589793, 0.0, 3.141592653589793], "texparams": ["\\\\pi", "0", "\\\\pi"], "qubits": [0]}, {"name": "measure", "qubits": [0], "clbits": [0], "memory": [0]}], "header": {"number_of_qubits": 3, "number_of_clbits": 3, "qubit_labels": [["q3", 0], ["q3", 1], ["q3", 2]], "clbit_labels": [["c3", 3]], "name": "circuit4", "compiled_circuit_qasm": "OPENQASM 2.0;\\ninclude \\"qelib1.inc\\";\\nqreg q3[3];\\ncreg c3[3];\\nmeasure q3[1] -> c3[1];\\nu3(3.14159265358979,0,3.14159265358979) q3[2];\\nmeasure q3[2] -> c3[2];\\nu3(3.